In [44]:
# Install the libraries (if using binder)
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install seaborn
# !pip install pylab

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pylab as plot

### Reading the csv file and get it into a dataframe format

In [46]:
df = pd.read_csv("pollution_us_2000_2016.csv")

In [47]:
print(f'The dataframe has {len(df)} rows and {df.shape[1]} columns')

The dataframe has 1746661 rows and 29 columns


Based on the above numbers, one concludes that the dataframe corresponds to very large dataset <em>(Big Data)</em>. Therefore, it is not advisable to view the entire dataframe as that will be a super memory expensive task. One needs to perform the analytics in a smarter way so as to the get the desired results while making sure not to put too much load on the memory.

In [82]:
states = df['State'].unique()
num_states = len(states)

In [83]:
print(f'There are {num_states} states in the dataset.')

There are 47 states in the dataset.


In [57]:
print(f'The columns in the dataframe are given by \n{df.columns}')

The columns in the dataframe are given by 
Index(['Unnamed: 0', 'State Code', 'County Code', 'Site Num', 'Address',
       'State', 'County', 'City', 'Date Local', 'NO2 Units', 'NO2 Mean',
       'NO2 1st Max Value', 'NO2 1st Max Hour', 'NO2 AQI', 'O3 Units',
       'O3 Mean', 'O3 1st Max Value', 'O3 1st Max Hour', 'O3 AQI', 'SO2 Units',
       'SO2 Mean', 'SO2 1st Max Value', 'SO2 1st Max Hour', 'SO2 AQI',
       'CO Units', 'CO Mean', 'CO 1st Max Value', 'CO 1st Max Hour', 'CO AQI'],
      dtype='object')


One would now like to get rid of the columns that are not going to be useful for our analytics.

In [58]:
# columns to be dropped
drop_cols = ['Unnamed: 0', 'State Code', 'County Code', 'Site Num', 'Address', 'City', 'NO2 1st Max Value', 'NO2 1st Max Hour', 'NO2 AQI', 'O3 1st Max Value', 'O3 1st Max Hour', 'O3 AQI', 'SO2 1st Max Value', 'SO2 1st Max Hour', 'SO2 AQI', 'CO 1st Max Value', 'CO 1st Max Hour', 'CO AQI']

In [59]:
df.drop(drop_cols, axis=1, inplace=True)

In [60]:
print(f'The dataframe now contains {len(df)} rows and {df.shape[1]} columns')

The dataframe now contains 1746661 rows and 11 columns



To get some idea about the entries in the dataframe, instad of looking at the entire dataframe, one peeks at only few rows.

In [61]:
df.head()

,State,County,Date Local,NO2 Units,NO2 Mean,O3 Units,O3 Mean,SO2 Units,SO2 Mean,CO Units,CO Mean
0,Arizona,Maricopa,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,3.000000,Parts per million,1.145833
1,Arizona,Maricopa,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,3.000000,Parts per million,0.878947
2,Arizona,Maricopa,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,2.975000,Parts per million,1.145833
3,Arizona,Maricopa,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,2.975000,Parts per million,0.878947
4,Arizona,Maricopa,2000-01-02,Parts per billion,22.958333,Parts per million,0.013375,Parts per billion,1.958333,Parts per million,0.850000


### Conversion of Units to achieve uniformity

By peeking at the above tiny dataframe, one sees that the units are not uniform for all the gases. Before converting all the units, one must make sure if all the entries in particular units column are the same, i.e., for a gas <em>x</em>, one would like to make sure if it is always measured in the same units.} 

In [62]:
def same_units(df, col):
    if len(df[col].unique()) == 1:
        return True
    return False

def get_units(df, col):
    return df[col].unique()

In [63]:
same_units(df, 'NO2 Units')

True

In [64]:
get_units(df, 'NO2 Units')

array(['Parts per billion'], dtype=object)

In [65]:
same_units(df, 'SO2 Units')

True

In [66]:
get_units(df, 'SO2 Units')

array(['Parts per billion'], dtype=object)

In [67]:
same_units(df, 'O3 Units')

True

In [68]:
get_units(df, 'O3 Units')

array(['Parts per million'], dtype=object)

In [69]:
same_units(df, 'CO Units')

True

In [70]:
get_units(df, 'CO Units')

array(['Parts per million'], dtype=object)


Note that, $NO_2$ and $SO_2$ are **always** measured in parts per billion (ppb), while $O_3$ and $CO$ are **always** measured in parts per million (ppm).

A standard unit in the field of science is parts per million (ppm), hence one would like to convert $NO_2$ and $SO_2$ into parts per million (ppm).

In [71]:
df['NO2 Mean'] = df['NO2 Mean']/1000
df['SO2 Mean'] = df['SO2 Mean']/1000


The columns describing the units are now irrelevant, hence, one would like to discard all of these columns to get even more finer dataframe.


In [72]:
unit_cols = ['NO2 Units', 'SO2 Units', 'O3 Units', 'CO Units']
df.drop(unit_cols, axis=1, inplace=True)

Before, moving ahead, a good practice is to check if there are any NaN, i.e., missing values in the dataframe. 

In [73]:
df.isna().any()

State         False
County        False
Date Local    False
NO2 Mean      False
O3 Mean       False
SO2 Mean      False
CO Mean       False
dtype: bool

Therefore, none of the above columns contain any missing value. Great!!

### Rename the 'Date Local' column and make it the index of the dataframe.

To see the trends with time, it is a good practice to have the date column as the index of the dataframe.

In [74]:
df.rename(columns={'Date Local':'Date'}, inplace=True)

In [75]:
# first check the type of the Date column
df['Date'].dtypes

dtype('O')

One would like to convert this into a 'DateTime Object' since that makes it easier to parse dates and do analysis on it.

In [76]:
df['Date'] = pd.to_datetime(df['Date'])

In [77]:
# make 'Date' as the index of the dataframe
df.set_index('Date', inplace=True)

In [78]:
# let us take a peek at the dataframe
df.tail()

,State,County,NO2 Mean,O3 Mean,SO2 Mean,CO Mean
Date,,,,,,
2016-03-30,Wyoming,Laramie,0.001083,0.043917,0.000000,0.091667
2016-03-31,Wyoming,Laramie,0.000939,0.045263,-0.000023,0.067714
2016-03-31,Wyoming,Laramie,0.000939,0.045263,-0.000023,0.100000
2016-03-31,Wyoming,Laramie,0.000939,0.045263,0.000000,0.067714
2016-03-31,Wyoming,Laramie,0.000939,0.045263,0.000000,0.100000


### Insert Total Mean Column

In order to quantify $NO_2$, $O_3$, $SO_2$ and $CO$ as one single entity for each state, one would like to get a **Total Mean** column, that allows us to do exactly that. The entries of this columns will act as a a measure for each state and will help us see how each of these states have performed through time.

In [79]:
df['Total Mean'] = df['NO2 Mean'] + df['SO2 Mean'] + df['CO Mean'] + df['O3 Mean']
df.head()

,State,County,NO2 Mean,O3 Mean,SO2 Mean,CO Mean,Total Mean
Date,,,,,,,
2000-01-01,Arizona,Maricopa,0.019042,0.022500,0.003000,1.145833,1.190375
2000-01-01,Arizona,Maricopa,0.019042,0.022500,0.003000,0.878947,0.923489
2000-01-01,Arizona,Maricopa,0.019042,0.022500,0.002975,1.145833,1.190350
2000-01-01,Arizona,Maricopa,0.019042,0.022500,0.002975,0.878947,0.923464
2000-01-02,Arizona,Maricopa,0.022958,0.013375,0.001958,0.850000,0.888292


### Get individual Dataframe for each of the Gases
The goal now is to create individual dataframes for each of the gases, with columns representing each of the states. One would also like to discard the states that have missing value for any of the years.

Note that, we the original dataset has 47 states (as computed before). It might be the case, that not all of them have values recorded for all the yeras from 2000 to 2016. We would like to get rid of them.

In [80]:
def get_df(gas_name):
    """
    
    """
    
    global df
    
    df_year_list = [df.loc[str(i)] for i in range(2000, 2017)]
    
    # populates dictionary with states as the keys and list of values
    dict_of_lists = {state:[] for state in states}
    
    for df_year in df_year_list:
        for state in states:
            state_vals = df_year[df_year['State'] == state]
            dict_of_lists[state].append(state_vals[gas_name].mean())
    
    # convert the dictionary to a dataframe
    result = pd.DataFrame(dict_of_lists)
    
    # change the indices so that they reflect years
    index_list = [item for item in range(2000, 2017)]
    result.index = index_list
    
    # get only the columns for which one has non-NaN values.
    result.dropna(axis = 1, inplace=True)
    return result

In [84]:
# dataframe for no2
no2_df = get_df('NO2 Mean')

In [85]:
# dataframe for so2
so2_df = get_df('SO2 Mean')

In [86]:
# dataframe for co
co_df = get_df('CO Mean')

In [87]:
# dataframe for o3
o3_df = get_df('O3 Mean')

In [88]:
# dataframe for 'Total Mean'
total_df = get_df('Total Mean')

Let us check one of the dataframes to see how it looks:

In [89]:
total_df

,Arizona,California,Colorado,District Of Columbia,Florida,Illinois,Indiana,Kansas,Louisiana,New York,North Carolina,Oklahoma,Pennsylvania,Texas
2000,0.798202,0.687464,0.645494,1.224127,0.765760,0.752500,0.925394,0.794784,0.801407,0.514070,0.671734,0.090925,0.348376,0.480331
2001,0.695511,0.666567,0.672161,1.127486,0.676223,0.640492,0.433128,0.374191,0.869818,0.547200,0.590789,0.076960,0.369254,0.428627
2002,0.754270,0.646514,0.541882,1.214099,0.506577,0.636477,0.499557,0.430260,0.757456,0.494625,0.484683,0.098953,0.308522,0.385840
2003,0.679421,0.618316,0.592142,0.946706,0.476274,0.559350,0.490029,0.355004,0.701848,0.581065,0.531398,0.085071,0.310436,0.504747
2004,0.554335,0.560905,0.657593,0.664519,0.556943,0.621316,0.613716,0.417344,0.603334,0.485550,0.473690,0.054292,0.368931,0.384332
2005,0.665032,0.500273,0.662426,0.895856,0.600239,0.503868,0.472980,0.423212,0.514256,0.451008,0.564364,0.059417,0.260185,0.357994
2006,0.603409,0.486094,0.618916,1.163044,0.427287,0.412583,0.704377,0.418019,0.413075,0.367027,0.450918,0.152595,0.276181,0.303200
2007,0.553691,0.456998,0.596887,1.245813,0.319266,0.366152,0.494955,0.463035,0.421861,0.412721,0.421601,0.241615,0.284100,0.267850
2008,0.470645,0.431742,0.452051,1.271644,0.257114,0.312789,0.460137,0.445292,0.320464,0.361006,0.364179,0.185803,0.227020,0.260187
2009,0.466621,0.434052,0.397771,0.912213,0.372236,0.301144,0.271358,0.336629,0.258601,0.281663,0.330452,0.254057,0.274872,0.233453


In [90]:
print(f'There are {len(total_df.columns)} states in the datframe corresponding to the total mean.')

There are 14 states in the datframe corresponding to the total mean.


Naturally, one would like to know if all the dataframes have same states.

In [93]:
print(total_df.columns == no2_df.columns)
print(no2_df.columns == o3_df.columns)
print(o3_df.columns == so2_df.columns)
print(so2_df.columns == co_df.columns)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True]


Therefore, only these 14 states have their data recorded from 2000 to 2016. Rest other states have missing data, hence are discareded in our analysis.

# Line Plots